In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, FixedLocator, FixedFormatter, LogLocator, NullFormatter, LogFormatter, EngFormatter, AutoMinorLocator)
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import struct
%matplotlib inline

# Defaults more suitable for 4K monitors
mpl.rcParams["figure.dpi"] = 200
mpl.rcParams["figure.figsize"] = [6.0, 4.0]

# Export settings
mpl.rcParams["savefig.bbox"] = 'tight'
mpl.rcParams["savefig.dpi"] = 300
mpl.rcParams["savefig.facecolor"] = 'white'

# Lossless using WebP
#mpl.rcParams["animation.codec"] = 'webp'
#mpl.rcParams["animation.ffmpeg_args"] = ["-preset", "drawing", "-lossless", "1"]
# Use anim.save("movie.webp", fps=60)

# Lossy using AV1
mpl.rcParams["animation.codec"] = 'libaom-av1'
mpl.rcParams["animation.ffmpeg_args"] = []
# Use anim.save("movie.webm", fps=60)


In [ ]:
import glob
import re

dfs = []
for path in glob.glob('./results/*/*/*.csv'):
    if 'skip' in path:
        continue
    m = re.match(r'.*/(.*)/(.*)/([^/]*)\.csv', path)
    (machine, algo, run) = (m.group(1), m.group(2), m.group(3))
    df = pd.read_csv(path)
    df['machine'] = machine
    df['algo'] = algo
    df['run'] = run
    dfs.append(df)
df = pd.concat(dfs)

In [ ]:
for machine in df['machine'].unique():

    fig, ax = plt.subplots()

    # Plot algos and runs for the machine
    dfm = df[df['machine'] == machine]
    min_y = 1e20
    max_y = 0.0
    for algo in dfm['algo'].unique():
        print(algo)
        dfa = dfm[dfm['algo'] == algo]
        for run in dfa['run'].unique():
            dfr = dfa[dfa['run'] == run]
            min_y = min(min_y, dfa['throughput'].min())
            max_y = max(max_y, dfa['throughput'].max())
            ax.plot(dfr['size'], dfr['throughput'], marker='.', alpha=1.0, markersize=3, markeredgewidth=0.7, linestyle='-', linewidth=0.5, label=algo+run)
        

    # Framing
    ax.set_title(machine)
    ax.set_clip_on(False)
    for s in ['top', 'right']:
        ax.spines[s].set_visible(False)

    # X axis
    ax.set_xlim((2**10, 2**32))
    ax.set_xscale('log')
    ax.xaxis.set_major_locator(FixedLocator([2**i for i in range(10,33,2)]))
    ax.xaxis.set_major_formatter(FixedFormatter(["2¹⁰", "2¹²", "2¹⁴", "2¹⁶", "2¹⁸", "2²⁰", "2²²", "2²⁴", "2²⁶", "2²⁸", "2³⁰", "2³²"]))
    ax.xaxis.set_minor_locator(FixedLocator([2**i for i in range(10,33)]))
    ax.xaxis.set_minor_formatter(NullFormatter())

    # Y axis
    ax.set_ylim((0.9 * min_y, 1.5 * max_y))
    # if machine == 'm1-max':
    #     ax.set_ylim((0.9 * min_y, 20e7))
    # if machine == 'hpc6a':
    #     ax.set_ylim((4e5, 4e7))
    # if machine == 'a15':
    #     ax.set_ylim((0.7e5, 1e7))
    ax.set_yscale('log')

    fig.legend(bbox_to_anchor=(0.5, 0.06), loc='upper center', ncol=4, frameon=False, markerscale=3)
    
    fig.savefig(f'bench-{machine}.png')
